In [1]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import os
import torch

2023-10-08 17:06:21,214 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-10-08 17:06:21,223 - modelscope - INFO - Loading ast index from /home/yangwenhao/.cache/modelscope/ast_indexer
2023-10-08 17:06:21,478 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 2fd7920946625a585615c128019784ec and a total number of 921 components indexed
/home/yangwenhao/anaconda3/envs/model/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# cam_pp
sv_pipeline = pipeline(
    task=Tasks.speaker_verification,
    model='damo/speech_campplus_sv_en_voxceleb_16k'
)

In [2]:
# ecapa-tdnn
sv_pipeline = pipeline(
    task='speaker-verification',
    model='damo/speech_ecapa-tdnn_sv_en_voxceleb_16k',
    device='cuda:1'
)

2023-10-08 17:06:26,047 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2023-10-08 17:06:26,589 - modelscope - INFO - initiate model from /home/yangwenhao/.cache/modelscope/hub/damo/speech_ecapa-tdnn_sv_en_voxceleb_16k
2023-10-08 17:06:26,590 - modelscope - INFO - initiate model from location /home/yangwenhao/.cache/modelscope/hub/damo/speech_ecapa-tdnn_sv_en_voxceleb_16k.
2023-10-08 17:06:26,592 - modelscope - INFO - initialize model from /home/yangwenhao/.cache/modelscope/hub/damo/speech_ecapa-tdnn_sv_en_voxceleb_16k


cuda:1


2023-10-08 17:06:33,584 - modelscope - WARNING - No preprocessor field found in cfg.
2023-10-08 17:06:33,586 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-10-08 17:06:33,587 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/yangwenhao/.cache/modelscope/hub/damo/speech_ecapa-tdnn_sv_en_voxceleb_16k'}. trying to build by task and model information.
2023-10-08 17:06:33,588 - modelscope - WARNING - No preprocessor key ('ecapa-tdnn-sv', 'speaker-verification') found in PREPROCESSOR_MAP, skip building preprocessor.


In [8]:
sv_pipeline.model 

SpeakerVerificationECAPATDNN(
  (embedding_model): ECAPA_TDNN(
    (blocks): ModuleList(
      (0): TDNNBlock(
        (conv): Conv1d(
          (conv): Conv1d(80, 1024, kernel_size=(5,), stride=(1,))
        )
        (activation): ReLU()
        (norm): BatchNorm1d(
          (norm): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): SERes2NetBlock(
        (tdnn1): TDNNBlock(
          (conv): Conv1d(
            (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
          )
          (activation): ReLU()
          (norm): BatchNorm1d(
            (norm): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (res2net_block): Res2NetBlock(
          (blocks): ModuleList(
            (0-6): 7 x TDNNBlock(
              (conv): Conv1d(
                (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), dilation=(2,))
              )
              (activation)

In [ ]:
from modelscope.pipelines import pipeline
sv_pipeline = pipeline(
    task='speaker-verification',
    model='damo/speech_campplus_sv_zh-cn_16k-common',
    model_revision='v1.0.0'
)

In [3]:
from tqdm import tqdm
import h5py
import torch
import soundfile as sf
from Eval.eval_metrics import evaluate_kaldi_eer, evaluate_kaldi_mindcf

def read_hdf5(reader, key):
    with h5py.File(reader, 'r') as r:
        data_flat = r.get(key)[:]
        return data_flat

In [4]:
test_subset = 'test_orgchn2dist3' # 'test_orgchn2dist1' 'test_orgchn2' # test_orgchn2band

save_dir    = 'radio/data/ecapa/radiodist3_2/' #
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
        
wav_scp     = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/wav.scp'%(test_subset)

uid2path = {}
emb_file = save_dir + 'xvector.h5py'

with  h5py.File(emb_file, 'w') as gf:
    with open(wav_scp, 'r') as f:
        for l in tqdm(f.readlines(), ncols=50):
            uid, upath = l.split()
            uid2path[uid] = upath
            wav, sr = sf.read(upath)
            if 'radchn2_dist1' in upath:
                wav = wav + 0.072346

            result = sv_pipeline(in_audios=[wav], output_emb=True)
            emb    = result[1]

            gf.create_dataset(uid, data=emb, compression="gzip")

100%|███████| 14622/14622 [03:57<00:00, 61.67it/s]


In [5]:
dist_fn = torch.nn.CosineSimilarity()

In [6]:
# for subs in ['bandpass', 'bandcross', 'bandradio']:
# for subs in ['original', 'radio', 'radiocross', 'bandpass', 'bandcross', 'bandradio']:
# for subs in ['rdist1', 'rdist1cross', 'radiodist1']:
for subs in ['rdist3', 'rdist3cross', 'radiodist3']:
    
    trials = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials_%s'%(test_subset, subs)
    distances = []
    labels = []
    with open(trials, 'r') as f:
        for l in tqdm(f.readlines(), ncols=50):
            a,b,target = l.split()
            a = torch.tensor(read_hdf5(emb_file, a))
            b = torch.tensor(read_hdf5(emb_file, b))
            d = dist_fn(a, b)
            distances.append(d[0])

            if target == 'target':
                labels.append(True)
            else:
                labels.append(False)

    eer, eer_threshold, accuracy = evaluate_kaldi_eer(
            distances, labels, cos=True, re_thre=True)
    mindcf_01, mindcf_001 = evaluate_kaldi_mindcf(distances, labels)

    print('{:<12s}:  EER(%): {:.2f}  MinDCF: {:.4f}'.format(subs, eer*100, mindcf_01))
    
# cam_pp vox2
# original    :  EER(%): 0.89  MinDCF: 0.0938
# radio       :  EER(%): 2.35  MinDCF: 0.3141
# radiocross  :  EER(%): 2.10  MinDCF: 0.2678

# ECAPA-TDNN vox2
# original    :  EER(%): 1.00  MinDCF: 0.1021
# radio       :  EER(%): 2.70  MinDCF: 0.3496
# radiocross  :  EER(%): 2.31  MinDCF: 0.3055
# bandpass    :  EER(%): 3.05  MinDCF: 0.3696
# bandcross   :  EER(%): 2.31  MinDCF: 0.3072
# bandradio   :  EER(%): 3.36  MinDCF: 0.4165

# dist_1 radio test
# rdist1      :  EER(%): 5.20  MinDCF: 0.4724
# rdist1cross :  EER(%): 4.35  MinDCF: 0.3779
# radiodist1  :  EER(%): 4.44  MinDCF: 0.4451

# dist_1 radio test wav_norm
# rdist1      :  EER(%): 5.20  MinDCF: 0.4724
# rdist1cross :  EER(%): 4.35  MinDCF: 0.3780
# radiodist1  :  EER(%): 4.44  MinDCF: 0.4451

# dist_1 radio test wav + const
# rdist1      :  EER(%): 5.20  MinDCF: 0.4723
# rdist1cross :  EER(%): 4.35  MinDCF: 0.3779
# radiodist1  :  EER(%): 4.44  MinDCF: 0.4451

# rdist3      :  EER(%): 6.18  MinDCF: 0.5804
# rdist3cross :  EER(%): 5.54  MinDCF: 0.4722
# radiodist3  :  EER(%): 5.41  MinDCF: 0.5143

# cam_pp 3d-zh
# original    :  EER(%): 7.88  MinDCF: 0.6422
# radio       :  EER(%): 21.27  MinDCF: 0.9945
# radiocross  :  EER(%): 24.14  MinDCF: 0.9264
# bandpass    :  EER(%): 17.62  MinDCF: 0.8550
# bandcross   :  EER(%): 17.66  MinDCF: 0.9662
# bandradio   :  EER(%): 26.63  MinDCF: 0.9773

100%|██████| 37720/37720 [00:50<00:00, 740.74it/s]


rdist3      :  EER(%): 6.18  MinDCF: 0.5804


100%|██████| 75440/75440 [01:45<00:00, 717.84it/s]


rdist3cross :  EER(%): 5.54  MinDCF: 0.4722


100%|██████| 75440/75440 [01:40<00:00, 748.85it/s]


radiodist3  :  EER(%): 5.41  MinDCF: 0.5143
